In [1]:
import cobra
from cobra.io import load_json_model, save_json_model
from cobra import Model, Reaction, Metabolite
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn3
from cobra.flux_analysis import gapfill

In [2]:
recon1 = load_json_model('../data/gems/RECON1.json')
icho = load_json_model('../data/gems/iCHOv1.json')
recon1_gene_dict = {gene.name: gene.id for gene in recon1.genes }
icho_gene_dict = {gene.name: gene.id for gene in icho.genes}
recon1_match = pd.read_csv('../data/blast_results/recon1_match.csv')
icho_match = pd.read_csv('../data/blast_results/icho_match.csv')

# Draft reconstruction

In [ ]:
def get_gene_states(rxn, match_table):
    output={}
    for gene in list(rxn.genes):
        if gene.name in list(match_table['query']):
            output[gene.id]='True'
        else:
            output[gene.id]='False'
    return output
            
def gpr_to_boolean(gpr_rule, gene_states):
    for gene, state in gene_states.items():
        gpr_rule = gpr_rule.replace(gene, str(state))
    return eval(gpr_rule) 

In [ ]:
genes_icho = list(np.unique(icho_match['query']))
# print(len(genes_icho))
genes_recon1 = list(np.unique(recon1_match['query']))
# print(len(genes_recon1 ))

rxns_icho, rxns_recon1 = [],[]
for g in genes_icho:
    for rxn in list( icho.genes.get_by_id( icho_gene_dict[g] ).reactions ):
        if gpr_to_boolean(rxn.gene_reaction_rule, get_gene_states(rxn, icho_match )):
            rxns_icho.append( rxn.id)
for g in genes_recon1:
    for rxn in list(recon1.genes.get_by_id( recon1_gene_dict[g] ).reactions ):
        if rxn.id in ['PPCOAOm','ACOAD1fm','ACOAD9m']:
            rxns_recon1.append(rxn.id) #special case
        elif gpr_to_boolean(rxn.gene_reaction_rule, get_gene_states(rxn, recon1_match )):
            rxns_recon1.append(rxn.id)
rxns_icho = list(set(rxns_icho)); rxns_recon1 = list(set(rxns_recon1))
print(len(rxns_icho))
print(len(rxns_recon1))

In [ ]:
fig,ax = plt.subplots(1,1)
venn2([set(rxns_icho),set(rxns_recon1)],set_labels=['iCHO matched','RECON1 matched'], ax=ax)
ax.set_title('Automatic curation of reactions')

In [ ]:
# ipsc_draft = load_json_model('../data/gems/iPSC_draft.json')

In [ ]:
recon1_match['template_gene'] = [ recon1_gene_dict[gname] for gname in list(recon1_match['query'])]
icho_match['template_gene'] = [icho_gene_dict[gname] for gname in list(icho_match['query']) ]

def add_metabolite(model, template, met_id):
    all_mets = [met.id for met in model.metabolites]
    new_id = met_id.replace('_cho','_psc').replace('_hs','_psc')
    if new_id not in all_mets:
        temp_met = template.metabolites.get_by_id( met_id )
        model.add_metabolites([Metabolite(id=new_id, formula=temp_met.formula,
                charge=float(temp_met.charge),name=temp_met.name, compartment=temp_met.compartment)])
    return None

def translate_gpr(template_rule, homology_table):
    tokens = template_rule.split()  
    translated_rule = []
    for token in tokens:
        if token not in list(homology_table['template_gene']):
            translated_rule.append(token)
        elif list(homology_table['template_gene']).count(token) == 1:
            new_token = list(homology_table[homology_table['template_gene']==token]['subject'])[0]
            translated_rule.append(new_token)
        elif list(homology_table['template_gene']).count(token) > 1:
            new_tokens = list(homology_table[homology_table['template_gene']==token]['subject'])
            new_token = ' or '.join(new_tokens)
            translated_rule.append(new_token)
    return ' '.join(translated_rule)
    

def add_rxn(model, template, match_table, rxn_id):
    all_rxns = [rxn.id for rxn in model.reactions]
    new_id = rxn_id.replace('_cho','').replace('_hs','')
    if new_id not in all_rxns:
        temp_rxn = template.reactions.get_by_id( rxn_id )
        model.add_reactions([Reaction(id=new_id,name=temp_rxn.name,subsystem= temp_rxn.subsystem,
                lower_bound=temp_rxn.lower_bound,upper_bound=temp_rxn.upper_bound)])
        for met in (temp_rxn.reactants+temp_rxn.products):
            add_metabolite(model, template, met.id)
        rxn_string = str(temp_rxn.reaction).replace('_cho','_psc').replace('_hs','_psc')
        model.reactions.get_by_id(new_id).build_reaction_from_string(rxn_string)
        model.reactions.get_by_id(new_id).gene_reaction_rule = translate_gpr(temp_rxn.gene_reaction_rule, match_table)
    return None

In [ ]:
# # add reactions from recon1
# for rxn_id in rxns_recon1:
#     add_rxn( ipsc_draft, recon1, recon1_match, rxn_id)
    
# # add reactions from icho
# for rxn_id in list( set(rxns_icho) - set(rxns_recon1) ):
#     add_rxn( ipsc_draft, icho, icho_match, rxn_id)

# Add boundary reactions

In [ ]:
# ipsc_draft = load_json_model('../data/gems/iPSC_draft.json')

In [ ]:
# for met in ipsc_draft.metabolites.query('_e'):
#     if met.compartment == 'e':
#         rxn = Reaction(id='EX_'+met.id,name=met.name+' exchange',subsystem= 'Extracellular exchange',
#                 lower_bound=-1000.0,upper_bound=1000.0)
#         ipsc_draft.add_reactions([rxn])
#         ipsc_draft.reactions.get_by_id( 'EX_'+met.id ).build_reaction_from_string(met.id + ' <=> ')

In [ ]:
# save_json_model(ipsc_draft,'../data/gems/iPSC_draft.json')

In [ ]:
# subsys_sum = {}
# for rxn in set(ibt_draft.reactions) - set(ibt_draft.boundary):
#     subsystem = rxn.subsystem
#     if ('Transport' in subsystem) or ('TRANSPORT' in subsystem):
#         continue
#     if subsystem == '':
#         subsystem = 'NaN'
#     if subsystem not in subsys_sum.keys():
#         subsys_sum[subsystem] = 1
#     else:
#         subsys_sum[subsystem] += 1   

In [ ]:
# sum_pd = pd.DataFrame( zip(subsys_sum.keys(), subsys_sum.values()) )
# sum_pd  = sum_pd.sort_values(by=1,ascending=False).reset_index().drop(['index'],axis=1)
# sum_top20 = sum_pd.iloc[:20]

In [ ]:
# fig, ax = plt.subplots(1,1,figsize=(10,8))
# ax.barh(sum_top20[0],sum_top20[1])
# ax.set_xlabel('# Reactions')
# ax.set_title('Top 20 subsystems')

# Gap fill with iCHO model

In [11]:
# with icho:
#     pfba_solution = cobra.flux_analysis.pfba(icho)
# pfba_solution.fluxes.to_csv('../data/fluxes_icho.csv')

In [27]:
def add_metabolite(model, template, met_id):
    all_mets = [met.id for met in model.metabolites]
    new_id = met_id.replace('_cho','_psc').replace('_hs','_psc')
    if new_id not in all_mets:
        temp_met = template.metabolites.get_by_id( met_id )
        model.add_metabolites([Metabolite(id=new_id, formula=temp_met.formula,
                charge=float(temp_met.charge),name=temp_met.name, compartment=temp_met.compartment)])
    return None
    

def add_reaction(model, template, rxn_id):
    all_rxns = [rxn.id for rxn in model.reactions]
    if rxn_id not in all_rxns:
        temp_rxn = template.reactions.get_by_id( rxn_id )
        new_id = rxn_id.replace('_cho','_psc').replace('_hs','_psc')
        model.add_reactions([Reaction(id=new_id,name=temp_rxn.name,
            subsystem= temp_rxn.subsystem, lower_bound=temp_rxn.lower_bound,upper_bound=temp_rxn.upper_bound)])
        for met in (temp_rxn.reactants+temp_rxn.products):
            add_metabolite(model, template, met.id)
        rxn_string = str(temp_rxn.reaction).replace('_cho','_psc').replace('_hs','_psc')
        model.reactions.get_by_id(new_id).build_reaction_from_string(rxn_string)
    return None

In [28]:
ipsc_draft =  load_json_model('../data/gems/iPSC_draft.json')
all_rxns = [rxn.id for rxn in ipsc_draft.reactions]

In [29]:
ex_comp = []
for rxn in (icho.reactions.query('tl')+icho.reactions.query('tm')+icho.reactions.query('tx')+\
           icho.reactions.query('tr')+icho.reactions.query('tn') + icho.reactions.query('tg') +\
           icho.reactions.query('tim')):
    if ('TRANSPORT' in rxn.subsystem) and (rxn.id not in all_rxns):
        ex_comp.append(rxn.id)
        add_reaction(ipsc_draft, icho, rxn.id)

In [30]:
GLDBRAN = Reaction(id='GLDBRAN',name='Glycogen debranching enzyme',subsystem= 'STARCH AND SUCROSE METABOLISM',
                lower_bound=0.0,upper_bound=1000.0)
SK_Tyr_ggn_c = Reaction(id='SK_Tyr_ggn_c',name='Sink for Tyr-194 Of Apo-Glycogenin Protein (Primer For Glycogen Synthesis)',
                subsystem= 'Intracellular source/sink', lower_bound=-0.1,upper_bound=1000.0)
r0954 = Reaction(id='r0954',name='MCT 1 Transport reaction',
                subsystem= 'TRANSPORT, PEROXISOMAL', lower_bound=-1000.0,upper_bound=1000.0)
CO2tx = Reaction(id='CO2tx',name='CO2 transporter, peroxisome',
                subsystem='TRANSPORT, PEROXISOMAL', lower_bound=-1000.0,upper_bound=1000.0)
O2tr = Reaction(id='O2tr',name='O2 transport endoplasmic reticulum',
                subsystem='TRANSPORT, ENDOPLASMIC RETICULAR', lower_bound=-1000.0,upper_bound=1000.0)
CO2tr = Reaction(id='CO2tr',name='CO2 endoplasmic reticular transport via diffusion',
                subsystem='TRANSPORT, ENDOPLASMIC RETICULAR', lower_bound=-1000.0,upper_bound=1000.0)
PGPP_psc = Reaction(id='PGPP_psc',name='Phosphatidylglycerol phosphate phosphatase (psc)',
                subsystem='GLYCEROPHOSPHOLIPID METABOLISM', lower_bound=0.0,upper_bound=1000.0)
MI3PS = Reaction(id='MI3PS',name='1D-myo-inositol-3-phosphate lyase (isomerizing)',
                subsystem='INOSITOL PHOSPHATE METABOLISM', lower_bound=-1000.0,upper_bound=1000.0)
ipsc_draft.add_reactions([GLDBRAN, SK_Tyr_ggn_c, r0954, CO2tx, O2tr, CO2tr, PGPP_psc, MI3PS])
ipsc_draft.reactions.GLDBRAN.build_reaction_from_string('dxtrn_c + h2o_c --> glc__D_c + glygn3_c')
ipsc_draft.reactions.SK_Tyr_ggn_c.build_reaction_from_string('Tyr_ggn_c <=> ')
ipsc_draft.reactions.r0954.build_reaction_from_string('mev__R_c <=> mev__R_x')
ipsc_draft.reactions.CO2tx.build_reaction_from_string('co2_c <=> co2_x')
ipsc_draft.reactions.CO2tr.build_reaction_from_string('co2_c <=> co2_r')
ipsc_draft.reactions.O2tr.build_reaction_from_string('o2_c <=> o2_r')
ipsc_draft.reactions.PGPP_psc.build_reaction_from_string('h2o_c + pgp_psc_c --> pglyc_psc_c + pi_c')
ipsc_draft.reactions.MI3PS.build_reaction_from_string('g6p_c <=> mi3p__D_c')

In [31]:
lipid_rxns =[]
for rxn in ( icho.reactions.query('ARTFR') + icho.reactions.query('ARTCOAL')+icho.reactions.query('ARTPLM') ) :
    lipid_rxns.append(rxn.id)
    add_reaction(ipsc_draft, icho, rxn.id)

In [32]:
EX_pydxn_e =  Reaction(id='EX_pydxn_e',name='Pyridoxine exchange',
                subsystem='Extracellular exchange', lower_bound=-1000.0,upper_bound=1000.0)
ipsc_draft.add_reactions([EX_pydxn_e])
ipsc_draft.reactions.EX_pydxn_e.build_reaction_from_string('pydxn_e <=> ')

In [52]:
gap_biomass = []
for met in ipsc_draft.reactions.BIOMASS_psc.reactants:
    with ipsc_draft:
        ipsc_draft.medium = {k:1000.0 for k in icho.medium.keys()}
        ipsc_draft.objective = ipsc_draft.add_boundary(met, type='demand')
        
        if ipsc_draft.slim_optimize() <= 0:
            gap_biomass.append(met.id)

In [53]:
gap_biomass

[]

In [49]:
with ipsc_draft:
    ipsc_draft.medium = {k:1000.0 for k in icho.medium.keys()}
    ipsc_draft.objective = ipsc_draft.add_boundary(ipsc_draft.metabolites.get_by_id('datp_c'), type='demand')
    if ipsc_draft.slim_optimize() > 0:
        print(1)

1


In [61]:
with ipsc_draft:
    ipsc_draft.medium = {k:1000.0 for k in icho.medium.keys()}
    print(ipsc_draft.medium )
    pfba_solution = cobra.flux_analysis.pfba(ipsc_draft)
    print(pfba_solution['BIOMASS_psc'])
#     pfba_solution.fluxes.to_csv('../data/fluxes_ipsc.csv')

{'EX_asn__L_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 1000.0, 'EX_cys__L_e': 1000.0, 'EX_gln__L_e': 1000.0, 'EX_hco3_e': 1000.0, 'EX_lys__L_e': 1000.0, 'EX_pro__L_e': 1000.0, 'EX_tyr__L_e': 1000.0, 'EX_glc__D_e': 1000.0, 'EX_met__L_e': 1000.0, 'EX_leu__L_e': 1000.0, 'EX_ser__L_e': 1000.0, 'EX_his__L_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_chol_e': 1000.0, 'EX_thr__L_e': 1000.0, 'EX_phe__L_e': 1000.0, 'EX_arg__L_e': 1000.0, 'EX_ile__L_e': 1000.0, 'EX_hxan_e': 1000.0, 'EX_val__L_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_trp__L_e': 1000.0, 'EX_fol_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_asp__L_e': 1000.0, 'EX_pydxn_e': 1000.0}
0.0


In [64]:
from cobra.medium import minimal_medium
temp = minimal_medium(ipsc_draft, 0.1, minimize_components=True)
print(temp)

/home/bras5181/miniconda3/lib/python3.8/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


EX_lys__L_e    0.038764
EX_pro__L_e    0.662520
EX_met__L_e    0.018867
EX_his__L_e    7.061128
EX_thr__L_e    1.604543
EX_ile__L_e    0.022017
EX_val__L_e    0.032263
EX_trp__L_e    0.002517
EX_atp_e       0.025487
EX_CE5786_e    0.042729
dtype: float64


In [66]:
ipsc_draft.metabolites.CE5786_e

Metabolite identifier,CE5786_e
Name,Kinetensin; Ile-Ala-Arg-Arg-His-Pro-Tyr-Phe-Leu
Memory address,0x07feec3135070
Formula,C56H87N17O11
Compartment,e
In 3 reaction(s),"RE2270E, EX_CE5786_e, RE2269E"


In [ ]:
# save_json_model(ipsc_draft,'../data/gems/iPSC_gf.json')